In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import tensorflow
gpus = tensorflow.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tensorflow.config.experimental.set_memory_growth(gpu, True)

In [4]:
%%capture
#@title
import os
import sys
import tensorflow.compat.v1 as tf

# Download source code.
if "efficientdet" not in os.getcwd():
  !git clone --depth 1 https://github.com/google/automl
  os.chdir('automl/efficientdet')
  sys.path.append('.')
  !pip install -r requirements.txt
  !pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
else:
  !git pull

In [5]:
MODEL = 'efficientdet-d7'  #@param

def download(m):
  if m not in os.listdir():
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{m}.tar.gz
    !tar zxf {m}.tar.gz
  ckpt_path = os.path.join(os.getcwd(), m)
  return ckpt_path

# Download checkpoint.
ckpt_path = download(MODEL)
print('Use model in {}'.format(ckpt_path))

# Prepare image and visualization settings.
image_url =  'https://user-images.githubusercontent.com/11736571/77320690-099af300-6d37-11ea-9d86-24f14dc2d540.png'#@param
image_name = 'img.png' #@param
!wget {image_url} -O img.png
import os
img_path = os.path.join(os.getcwd(), 'img.png')

min_score_thresh = 0.35  #@param
max_boxes_to_draw = 200  #@param
line_thickness =   2#@param

import PIL
# Get the largest of height/width and round to 128.
image_size = max(PIL.Image.open(img_path).size)

--2024-07-01 05:46:33--  https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/efficientdet-d7.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.195.207, 142.251.184.207, 172.217.214.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.195.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 413468440 (394M) [application/x-tar]
Saving to: 'efficientdet-d7.tar.gz'

efficientdet-d7.tar 100%[===================>] 394.31M   249MB/s    in 1.6s    

2024-07-01 05:46:35 (249 MB/s) - 'efficientdet-d7.tar.gz' saved [413468440/413468440]

Use model in /kaggle/working/automl/efficientdet/efficientdet-d7
--2024-07-01 05:46:41--  https://user-images.githubusercontent.com/11736571/77320690-099af300-6d37-11ea-9d86-24f14dc2d540.png
Resolving user-images.githubusercontent.com (user-images.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to user-images.githubuserconten

In [6]:
# # Get pascal voc 2012 trainval data
# import os
# if 'VOCdevkit' not in os.listdir():
#   !wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
#   !tar xf VOCtrainval_06-Nov-2007.tar

#   !mkdir tfrecord
#   !python -m dataset.create_pascal_tfrecord  \
#     --data_dir=VOCdevkit --year=VOC2007  --output_path=tfrecord/pascal

# # Pascal has 5717 train images with 100 shards epoch, here we use a single shard
# # for demo, but users should use all shards pascal-*-of-00100.tfrecord.
# file_pattern = 'pascal-00000-of-00100.tfrecord'  # @param
# images_per_epoch = 57 * len(tf.io.gfile.glob('tfrecord/' + file_pattern))
# images_per_epoch = images_per_epoch // 8 * 8  # round to 64.
# print('images_per_epoch = {}'.format(images_per_epoch))

import os
!cd automl/efficientdet/dataset
!curl -L "https://app.roboflow.com/ds/H96YBtcnAw?key=4jWyU34taw" > roboflow.zip
!unzip roboflow.zip -d dataset
!rm roboflow.zip

# # Count the number of images
# dataset_dir = 'dataset'
# batch_size = 16
# num_images = len([name for name in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir, name)) and name.endswith('.jpg')])

# # Set images per epoch
# images_per_epoch = (num_images // batch_size) * batch_size
# print('Images per epoch:', images_per_epoch)
# Since you know the total number of images
total_images = 1566

# Round to the nearest multiple of 64
images_per_epoch = total_images //8

print('images_per_epoch = {}'.format(images_per_epoch))


/bin/bash: line 0: cd: automl/efficientdet/dataset: No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   905  100   905    0     0   4517      0 --:--:-- --:--:-- --:--:--  4525
100 93.6M  100 93.6M    0     0  98.6M      0 --:--:-- --:--:-- --:--:--  159M
Archive:  roboflow.zip
  inflating: dataset/README.dataset.txt  
  inflating: dataset/README.roboflow.txt  
   creating: dataset/test/
 extracting: dataset/test/visual-pollutants.tfrecord  
  inflating: dataset/test/visual-pollutants_label_map.pbtxt  
   creating: dataset/train/
 extracting: dataset/train/visual-pollutants.tfrecord  
  inflating: dataset/train/visual-pollutants_label_map.pbtxt  
images_per_epoch = 195


In [ ]:

# generating train tfrecord is large, so we skip the execution here.
import os
if MODEL not in os.listdir():
  download(MODEL)


!mkdir /tmp/model_dir/
# key option: use --ckpt rather than --backbone_ckpt.
# Run training script with specified arguments



!python -m tf2.train --mode=traineval \
    --train_file_pattern=dataset/train/visual-pollutants.tfrecord \
    --val_file_pattern=dataset/test/visual-pollutants.tfrecord\
    --model_name={MODEL} \
    --model_dir=/tmp/model_dir/{MODEL}-finetune \
    --pretrained_ckpt={MODEL} \
    --batch_size=8 \
    --eval_samples={images_per_epoch}  \
    --num_examples_per_epoch={images_per_epoch}  --num_epochs=100  \
    --hparams="num_classes=5,moving_average_decay=0,mixed_precision=true"



